# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Oct 7 2022 2:15PM,248210,10,8622951,"ClearSpec, LLC",Released
1,Oct 7 2022 1:51PM,248207,10,0086064831,ISDIN Corporation,Released
2,Oct 7 2022 1:51PM,248207,10,0086064835,ISDIN Corporation,Released
3,Oct 7 2022 1:51PM,248207,10,0086064840,ISDIN Corporation,Released
4,Oct 7 2022 1:51PM,248207,10,0086064866,ISDIN Corporation,Released
5,Oct 7 2022 1:51PM,248207,10,0086064877,ISDIN Corporation,Released
6,Oct 7 2022 1:51PM,248207,10,0086064893,ISDIN Corporation,Released
7,Oct 7 2022 1:51PM,248207,10,0086064901,ISDIN Corporation,Released
8,Oct 7 2022 1:51PM,248207,10,0086064931,ISDIN Corporation,Released
9,Oct 7 2022 1:51PM,248207,10,0086064930,ISDIN Corporation,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
31,248204,Released,18
32,248206,Executing,2
33,248206,Released,4
34,248207,Released,12
35,248210,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Cancelled,Completed,Executing,Released
id,,,,
248203,NaN,NaN,NaN,1.0
248204,NaN,NaN,NaN,18.0
248206,NaN,NaN,2.0,4.0
248207,NaN,NaN,NaN,12.0
248210,NaN,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Cancelled,Completed,Executing,Released
id,,,,
248074,0.0,17.0,9.0,16.0
248115,0.0,0.0,5.0,4.0
248117,0.0,0.0,0.0,5.0
248119,0.0,0.0,0.0,6.0
248120,0.0,0.0,0.0,5.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Cancelled,Completed,Executing,Released
id,,,,
248074,0.0,17,9,16
248115,0.0,0,5,4
248117,0.0,0,0,5
248119,0.0,0,0,6
248120,0.0,0,0,5


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Cancelled,Completed,Executing,Released
0,248074,0.0,17,9,16
1,248115,0.0,0,5,4
2,248117,0.0,0,0,5
3,248119,0.0,0,0,6
4,248120,0.0,0,0,5


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Cancelled,Completed,Executing,Released
0,248074,0.0,17,9,16
1,248115,0.0,,5,4
2,248117,0.0,,,5
3,248119,0.0,,,6
4,248120,0.0,,,5


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id             int64
Cancelled    float64
Completed     object
Executing     object
Released      object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Oct 7 2022 2:15PM,248210,10,"ClearSpec, LLC"
1,Oct 7 2022 1:51PM,248207,10,ISDIN Corporation
13,Oct 7 2022 1:50PM,248206,19,"AdvaGen Pharma, Ltd"
19,Oct 7 2022 1:47PM,248204,10,ISDIN Corporation
37,Oct 7 2022 1:45PM,248203,19,ACG North America LLC
38,Oct 7 2022 1:40PM,248202,20,"Adare Pharmaceuticals, Inc."
39,Oct 7 2022 1:28PM,248193,15,"Person & Covey, Inc."
54,Oct 7 2022 1:27PM,248199,19,Eli Lilly and Company
55,Oct 7 2022 1:25PM,248192,15,"Person & Covey, Inc."
70,Oct 7 2022 1:24PM,248191,15,"Person & Covey, Inc."


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Cancelled,Completed,Executing,Released
0,Oct 7 2022 2:15PM,248210,10,"ClearSpec, LLC",0.0,,,1
1,Oct 7 2022 1:51PM,248207,10,ISDIN Corporation,0.0,,,12
2,Oct 7 2022 1:50PM,248206,19,"AdvaGen Pharma, Ltd",0.0,,2,4
3,Oct 7 2022 1:47PM,248204,10,ISDIN Corporation,0.0,,,18
4,Oct 7 2022 1:45PM,248203,19,ACG North America LLC,0.0,,,1
5,Oct 7 2022 1:40PM,248202,20,"Adare Pharmaceuticals, Inc.",0.0,,,1
6,Oct 7 2022 1:28PM,248193,15,"Person & Covey, Inc.",0.0,,,15
7,Oct 7 2022 1:27PM,248199,19,Eli Lilly and Company,0.0,,,1
8,Oct 7 2022 1:25PM,248192,15,"Person & Covey, Inc.",0.0,,,15
9,Oct 7 2022 1:24PM,248191,15,"Person & Covey, Inc.",0.0,,,15


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Oct 7 2022 2:15PM,248210,10,"ClearSpec, LLC",1,,
1,Oct 7 2022 1:51PM,248207,10,ISDIN Corporation,12,,
2,Oct 7 2022 1:50PM,248206,19,"AdvaGen Pharma, Ltd",4,2,
3,Oct 7 2022 1:47PM,248204,10,ISDIN Corporation,18,,
4,Oct 7 2022 1:45PM,248203,19,ACG North America LLC,1,,
5,Oct 7 2022 1:40PM,248202,20,"Adare Pharmaceuticals, Inc.",1,,
6,Oct 7 2022 1:28PM,248193,15,"Person & Covey, Inc.",15,,
7,Oct 7 2022 1:27PM,248199,19,Eli Lilly and Company,1,,
8,Oct 7 2022 1:25PM,248192,15,"Person & Covey, Inc.",15,,
9,Oct 7 2022 1:24PM,248191,15,"Person & Covey, Inc.",15,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Oct 7 2022 2:15PM,248210,10,"ClearSpec, LLC",1,,
1,Oct 7 2022 1:51PM,248207,10,ISDIN Corporation,12,,
2,Oct 7 2022 1:50PM,248206,19,"AdvaGen Pharma, Ltd",4,2,
3,Oct 7 2022 1:47PM,248204,10,ISDIN Corporation,18,,
4,Oct 7 2022 1:45PM,248203,19,ACG North America LLC,1,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

C:\Users\jmeza.WOODFIELD\.conda\envs\PythonData\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\jmeza.WOODFIELD\.conda\envs\PythonData\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\jmeza.WOODFIELD\.conda\envs\PythonData\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Oct 7 2022 2:15PM,248210,10,"ClearSpec, LLC",1.0,NaN,NaN
1,Oct 7 2022 1:51PM,248207,10,ISDIN Corporation,12.0,NaN,NaN
2,Oct 7 2022 1:50PM,248206,19,"AdvaGen Pharma, Ltd",4.0,2.0,NaN
3,Oct 7 2022 1:47PM,248204,10,ISDIN Corporation,18.0,NaN,NaN
4,Oct 7 2022 1:45PM,248203,19,ACG North America LLC,1.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Oct 7 2022 2:15PM,248210,10,"ClearSpec, LLC",1.0,0.0,0.0
1,Oct 7 2022 1:51PM,248207,10,ISDIN Corporation,12.0,0.0,0.0
2,Oct 7 2022 1:50PM,248206,19,"AdvaGen Pharma, Ltd",4.0,2.0,0.0
3,Oct 7 2022 1:47PM,248204,10,ISDIN Corporation,18.0,0.0,0.0
4,Oct 7 2022 1:45PM,248203,19,ACG North America LLC,1.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,1737322,35.0,14.0,1.0
15,992650,61.0,9.0,17.0
16,2977636,40.0,9.0,0.0
18,248172,0.0,0.0,0.0
19,744608,6.0,2.0,0.0
20,496370,2.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,1737322,35.0,14.0,1.0
1,15,992650,61.0,9.0,17.0
2,16,2977636,40.0,9.0,0.0
3,18,248172,0.0,0.0,0.0
4,19,744608,6.0,2.0,0.0
5,20,496370,2.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,35.0,14.0,1.0
1,15,61.0,9.0,17.0
2,16,40.0,9.0,0.0
3,18,0.0,0.0,0.0
4,19,6.0,2.0,0.0
5,20,2.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,35.0
1,15,Released,61.0
2,16,Released,40.0
3,18,Released,0.0
4,19,Released,6.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,15,16,18,19,20
Status,,,,,,
Completed,1.0,17.0,0.0,0.0,0.0,0.0
Executing,14.0,9.0,9.0,0.0,2.0,0.0
Released,35.0,61.0,40.0,0.0,6.0,2.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,15,16,18,19,20
0,Completed,1.0,17.0,0.0,0.0,0.0,0.0
1,Executing,14.0,9.0,9.0,0.0,2.0,0.0
2,Released,35.0,61.0,40.0,0.0,6.0,2.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,15,16,18,19,20
0,Completed,1.0,17.0,0.0,0.0,0.0,0.0
1,Executing,14.0,9.0,9.0,0.0,2.0,0.0
2,Released,35.0,61.0,40.0,0.0,6.0,2.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()